<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/provide_your_own_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>No model WebUI Colab</code></h2>

### 🤔 How to use?

 1. Download the models you want in the first cell, make sure the link you provide is the direct link to the model.
 2. Go to 🚀 `Launch web UI` to get any VAEs (or not) and wait.

If things breaks, call my attention by opening an issue at this [Github Repository](https://github.com/NUROISEA/anime-webui-colab)

In [ ]:
#@title ⏬ Download models

#@markdown You can run this multiple times

#@markdown I recommend downloading from HuggingFace! :)

#@markdown Direct model link
model_link = "https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.safetensors" #@param { type: 'string' }

#@markdown Does this model have a `.yaml` file?
model_has_yaml = False #@param { type: 'boolean' }
model_yaml_link = "" #@param { type: 'string' }

#@markdown Direct VAE link
vae_link = "https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt" #@param { type: 'string' }

models_folder = "/content/models"
vae_folder = "/content/VAE"

git_flags = "-q"
wget_flags = "-q --show-progress"

has_downloaded_at_least_once = True

try:
  model_list
  vae_list
except NameError:
  model_list = []
  vae_list = []

if model_link is not "":
  model_name = model_link.split("/")[-1]
  model_list.append(model_name)

  !wget {wget_flags} {model_link} -O {models_folder}/{model_name}
  
  if model_has_yaml and model_yaml_link is not "":
    !wget {wget_flags} {model_link} -O {models_folder}/{model_name}.yaml
else:
  print("No model link entered. Downloading nothing.")

if vae_link is not "":
  vae_name = vae_link.split("/")[-1]

  vae_list.append(vae_name)
  
  !wget {wget_flags} {vae_link} -O {vae_folder}/{vae_name}
else:
  print("No VAE link entered. Downloading nothing.")

print("Downloading done.")

model_listing = "\n -".join(model_list)
vae_listing = "\n -".join(vae_list)

print("List of models downloaded:")
print(model_lising)

print("List of VAEs downloaded:")
print(vae_listing)

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown Select a model to forefully load
model = "" #@param { type: 'string' }

#@markdown Select a VAE to forefully load
vae = "" #@param { type: 'string' }

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.app` link to the web UI

try:
  has_downloaded_at_least_once
except:
  sys.exit("You haven't dowloaded anything, please run the previous cell first!")

try:
  has_run
except NameError:
  has_run = False

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"

if not has_run:
  has_run = True
  
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  
  !git clone {git_flags} https://github.com/camenduru/stable-diffusion-webui
  !git clone {git_flags} https://github.com/yfszzx/stable-diffusion-webui-images-browser {extensions_folder}/stable-diffusion-webui-images-browser
  !git clone {git_flags} https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {extensions_folder}/a1111-sd-webui-tagcomplete
  
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/anything_ui_config.json -O {web_ui_folder}/ui-config.json
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/config.json -O {web_ui_folder}/config.json
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/styles.csv -O {web_ui_folder}/styles.csv

%cd {web_ui_folder}

arg_list = [
  "--share --xformers"
]

if model is not "":
  arg_list.append(f"--ckpt {models_folder}/{model}")

if vae is not "":
  arg_list.append(f"--vae-path {vae_folder}/{vae_name}")

args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 📩 `Saving images`

#@markdown The zip file will be visible at the files tab.

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"Zipped. You can now find {archive_name} at the files tab.")


#@markdown 📤 Upload zip to [temp.sh](https://temp.sh/)?
upload_to_tempsh = False #@param {type:'boolean'}
if upload_to_tempsh:
  !curl -T /content/{archive_name} temp.sh
  print("\n^^^ This is your download link! ^^^")

#@markdown ----

#@markdown 📲 Copy zip to Google Drive?
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"Copied to {gdrive_folder}!")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = True #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs